In [1]:
import keras

Using TensorFlow backend.


In [2]:
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
    def update(self, layer):
        grads = layer.grads
        for key in layer.params.keys():
            #print('勾配：', grads[key])
            layer.params[key] -= self.lr * grads[key]
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """

In [3]:
class ConvSimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma):
        self.sigma = sigma
        self.filter_num = None
        
    def W(self, filter_num, filter_length):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W :
        """
        W = np.random.randn(filter_num, filter_length) * self.sigma
        self.filter_num = filter_num
        return W
    
    def B(self):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :
        """
        
        B = np.random.randn(self.filter_num,)
        return B

# 問題1,問題2,問題3

In [4]:
class SimpleConv1d:
    
    def __init__(self,filter_num, filter_length,
                 initializer, optimizer,sigma,
                 opt_params, stride=1, padding=0):
        #self.input_shape= input_shape
        self.filter_num = filter_num
        self.filter_length = filter_length
        print(self.filter_length)
        self.stride = stride
        self.padding = padding
        self.sigma = sigma
        self.initializer = initializer(self.sigma)
        self.optimizer = optimizer(**opt_params)
        
        
        self.params = {}
        self.grads = {}
        self.X = None
        self.out_size = None
        #self.params['W'] = self.initializer.W(self.filter_num, self.filter_length)
        self.params['W'] = np.array([3,5,7]).reshape(1,-1)
        #self.params['B'] = self.initializer.B()
        self.params['B'] = np.array([1])
        self.dW = np.zeros_like(self.params['W'])
        self.dB = np.zeros_like(self.params['B'])
        self.grads['W'] = self.dW
        self.grads['B'] = self.dB
        
        
    def forward(self,X):
        self.X = X
        self.dX = np.zeros_like(X)
        D = len(X)
        #N,D = X.shape
        self.out_size = self.calc_out_size(D, self.filter_length, self.padding,
                                    self.stride)
        print('出力サイズ：',self.out_size)
        x_out = np.zeros((self.out_size,self.filter_num))
        #x_out = np.zeros((N, out_size,self.filter_num))
        count = 0
        for i in range(self.out_size):
            x_out[i,:] = np.dot(X[i : i + self.filter_length], self.params['W'].T) + \
            self.params['B']
            #x_out[:, i, :] = np.dot(X[:,i : self.filter_length+ i], self.params['W'].T) + \
            #self.params['B']#shape=(N,FN)
            #print('一つの領域', x_out[:,i,:])
            count += 1
        
        print('count',count)
        self.x_out = x_out
        return x_out
    
    
    def backward(self, dout):
        #shape: dout=(1,FN), x=(1,filter_size), W=(FN, filter_size)
        for i in range(self.out_size):
            print(dout[i,:].reshape(1,-1).shape,'dout[i,:].T.shape')
            print(self.X[i : i + self.filter_length].reshape(1,-1).shape,
                  'xshape')
            print(self.grads['W'].shape, 'Wshape')
            self.grads['W'] += np.dot(dout[i,:].T.reshape(1,-1),
                                      self.X[i : i + self.filter_length].reshape(1,-1))
            self.grads['B'] += dout[i,:]
            self.dX[i : i + self.filter_length] += np.dot(dout[i,:], 
                                                          self.params['W'])
            
        self.grads['W'] = self.grads['W'] #/ self.out_size
        self.grads['B'] = self.grads['B']# / self.out_size
        self.dX = self.dX #/ self.out_size
        return self.dX 
        
        
        
    
    def calc_out_size(self, D, filter_length, padding, stride):
        return (D - filter_length + 2 * padding) // stride + 1
            
                
        
        
        
        

In [6]:
import numpy as np
filter_num = 1
filter_length = 3
sigma = 0.01
X_sample = np.array([1,2,3,4])


conv1d = SimpleConv1d(filter_num=filter_num,
                      filter_length=filter_length, sigma = sigma,
                      initializer=ConvSimpleInitializer, optimizer=SGD,
                 opt_params={'lr' : 0.01})
out = conv1d.forward(X_sample)
print(out)

3
出力サイズ： 2
count 2
[[35.]
 [50.]]


In [33]:
out.shape

(2, 1)

In [7]:
delta_a = np.array([10,20]).reshape(-1,1)
dout = conv1d.backward(delta_a)

(1, 1) dout[i,:].T.shape
(1, 3) xshape
(1, 3) Wshape
(1, 1) dout[i,:].T.shape
(1, 3) xshape
(1, 3) Wshape


In [8]:
print(dout)
print(conv1d.grads)

[ 30 110 170 140]
{'W': array([[ 50,  80, 110]]), 'B': array([30])}


In [9]:
class SimpleConv1d_ver1_2:
    
    def __init__(self,filter_num, filter_length,
                 initializer, optimizer,sigma,
                 opt_params, stride=1, padding=0):
        #self.input_shape= input_shape
        self.filter_num = filter_num
        self.filter_length = filter_length
        print(self.filter_length)
        self.stride = stride
        self.padding = padding
        self.sigma = sigma
        self.initializer = initializer(self.sigma)
        self.optimizer = optimizer(**opt_params)
        
        
        self.params = {}
        self.grads = {}
        self.X = None
        self.out_size = None
        #self.params['W'] = self.initializer.W(self.filter_num, self.filter_length)
        
        #Wを縦ベクトルで定義する
        self.params['W'] = np.array([3,5,7]).reshape(-1,1)
        #self.params['B'] = self.initializer.B()
        self.params['B'] = np.array([1])
        self.dW = np.zeros_like(self.params['W'])
        self.dB = np.zeros_like(self.params['B'])
        self.grads['W'] = self.dW
        self.grads['B'] = self.dB
        
        
    def forward(self,X):
        if X.ndim == 1:
            X = X.reshape(1,-1)
        
        N,D = X.shape
        self.X = X
        self.out_size = self.calc_out_size(D, self.filter_length,
                                           self.padding,self.stride)
        print('out_size',self.out_size)

        self.col = np.zeros((self.out_size, self.filter_length))

        #Xをcolに変換
        for i in range(self.out_size):
            self.col[i,:] = X[:, i : i + self.filter_length]
        
        out = np.dot(self.col, self.params['W']) + self.params['B']
        return out
        
    
    
    def backward(self, dout):
        #shape: dout=(out_size,FN), col=(out_size,filter_length),
        #W=(FN, filter_length)
        
        col_dX = np.dot(dout, self.params['W'].T)
        dX = np.zeros_like(self.X).astype(np.float64)
        print('dxshape',dX.shape)
        self.grads['W'] = np.dot(self.col.T, dout)
        self.grads['B'] = np.sum(dout,axis=0)# / self.out_size
        print('beforedx:', dX)
        for i in range(self.out_size):
            dX[:,i : i + self.filter_length] += col_dX[i,:]
            print(str(i) + '回目dX:', dX)
     
        return dX 
        
        
        
    
    def calc_out_size(self, D, filter_length, padding, stride):
        return (D - filter_length + 2 * padding) // stride + 1
            
                
        
        
        
        

In [10]:
filter_num = 1
filter_length = 3
sigma = 0.01
X_sample = np.array([1,2,3,4])


conv1d = SimpleConv1d_ver1_2(filter_num=filter_num,
                      filter_length=filter_length, sigma = sigma,
                      initializer=ConvSimpleInitializer, optimizer=SGD,
                 opt_params={'lr' : 0.01})
out = conv1d.forward(X_sample)
print(out)

delta_a = np.array([10,20]).reshape(-1,1)
dout = conv1d.backward(delta_a)



print(dout)

3
out_size 2
[[35.]
 [50.]]
dxshape (1, 4)
beforedx: [[0. 0. 0. 0.]]
0回目dX: [[30. 50. 70.  0.]]
1回目dX: [[ 30. 110. 170. 140.]]
[[ 30. 110. 170. 140.]]


# 問題4 チャンネル数を限定しない1次元畳み込み層クラスの作成

In [11]:
class SimpleConv1d_ver2_1:
    
    def __init__(self,filter_num, filter_length,
                 initializer, optimizer,sigma,
                 opt_params, stride=1, padding=0):
        #self.input_shape= input_shape
        self.filter_num = filter_num
        self.filter_length = filter_length
        print(self.filter_length)
        self.stride = stride
        self.padding = padding
        self.sigma = sigma
        self.initializer = initializer(self.sigma)
        self.optimizer = optimizer(**opt_params)
        
        
        self.params = {}
        self.grads = {}
        self.X = None
        self.col = None
        self.col_W = None
        self.out_size = None
        #self.params['W'] = self.initializer.W(self.filter_num, self.filter_length)
        
        #Wを縦ベクトルで定義する
        self.params['W'] = np.array([[[1,1,2],[2,1,1]],
                                   [[2,1,1],[1,1,1]],
                                   [[1,1,1],[1,1,1]]])#(FN,C,filter_length)
        
        #self.params['B'] = self.initializer.B()
        self.params['B'] = np.array([1, 2, 3]).reshape(1,-1)
        self.dW = np.zeros_like(self.params['W'])
        self.dB = np.zeros_like(self.params['B'])
        self.grads['W'] = self.dW
        self.grads['B'] = self.dB
        
        
    def forward(self,X):
        #batchsize,channel共に１のとき
#         if X.ndim == 1:
#             X = X[np.newaxis,np.newaxis,:]
#         #batchsizeが１のとき
#         if X.ndim == 2:
#             X = X[np.newaxis,:,:]
            
        C,D = X.shape
        self.C = C
        self.X = X
        self.out_size = self.calc_out_size(D, self.filter_length,
                                           self.padding,self.stride)
        print('out_size',self.out_size)
        
        #Wをcolに変換
        print('変換前：', self.params['W'])
        col_W = self.params['W'].transpose(1,2,0)
        col_W = col_W.reshape(-1,self.filter_num)
        self.col_W = col_W
        print('col_W', col_W)

        self.col = np.zeros((self.out_size, C * self.filter_length))
        print('初期化時colshape', self.col.shape)

        #Xをcolに変換
        for i in range(self.out_size):
            print('x部分shape', X[:, i : i + self.filter_length].shape)
            self.col[i,:] = X[:, i : i + self.filter_length].reshape(1,-1)
            print(str(i) + '番目のcol', self.col)
            
        print('colshape:', self.col.shape)
        out_col = np.dot(self.col, col_W) + \
        self.params['B']
        out = out_col.T
        return out#(FN,out_size)
        
    
    
    def backward(self, dout):
        #shape: dout=(out_size,FN), col=(out_size,filter_length),
        #W=(FN, filter_length)
        dout = dout.T#(out_size, FN)
        print('dout:', dout)
        print('col_W.T', self.col_W.T)
        col_dX = np.dot(dout, self.col_W.T)
        print('col_dX:', col_dX)
        #(out_size,FN)dot(FN,filter_length*C)
        dX = np.zeros_like(self.X).astype(np.float64)
        print('dxshape',dX.shape)
        #(filter_length * C, out_size)dot(out_size, FN)
        self.grads['W'] = np.dot(self.col.T, dout)
        self.grads['B'] = np.sum(dout,axis=0)
        print('beforedx:', dX)
        
        #col_dXをdXに変換
        col_dX_3d = col_dX.reshape(self.out_size, self.C, self.filter_length)
        col_dX_3d = col_dX_3d.transpose(1,2,0)
        for i in range(self.out_size):
            dX[:,i : i + self.filter_length] += col_dX_3d[:,:,i]
            print(str(i) + '回目dX:', col_dX_3d[:,:,i])
     
        return dX 
        
        
        
    
    def calc_out_size(self, D, filter_length, padding, stride):
        return (D - filter_length + 2 * padding) // stride + 1

In [12]:
conv1d.params['W'].shape

(3, 1)

In [13]:
#x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
#w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
#b = np.array([1, 2, 3])

import numpy as np
filter_num = 3
filter_length = 3
sigma = 0.01

print(type(np.array([[1, 2, 3, 4], [2, 3, 4, 5]])))
X_sample =np.array([[1, 2, 3, 4],[2, 3, 4, 5]])


conv1d = SimpleConv1d_ver2_1(filter_num=filter_num,
                      filter_length=filter_length, sigma = sigma,
                      initializer=ConvSimpleInitializer, optimizer=SGD,
                 opt_params={'lr' : 0.01})
out = conv1d.forward(X_sample)
print('out', out)

delta_a  = np.array([[9, 11],[32, 35],[52, 56]])
dout = conv1d.backward(delta_a)



print('dout', dout)

<class 'numpy.ndarray'>
3
out_size 2
変換前： [[[1 1 2]
  [2 1 1]]

 [[2 1 1]
  [1 1 1]]

 [[1 1 1]
  [1 1 1]]]
col_W [[1 2 1]
 [1 1 1]
 [2 1 1]
 [2 1 1]
 [1 1 1]
 [1 1 1]]
初期化時colshape (2, 6)
x部分shape (2, 3)
0番目のcol [[1. 2. 3. 2. 3. 4.]
 [0. 0. 0. 0. 0. 0.]]
x部分shape (2, 3)
1番目のcol [[1. 2. 3. 2. 3. 4.]
 [2. 3. 4. 3. 4. 5.]]
colshape: (2, 6)
out [[21. 29.]
 [18. 25.]
 [18. 24.]]
dout: [[ 9 32 52]
 [11 35 56]]
col_W.T [[1 1 2 2 1 1]
 [2 1 1 1 1 1]
 [1 1 1 1 1 1]]
col_dX: [[125  93 102 102  93  93]
 [137 102 113 113 102 102]]
dxshape (2, 4)
beforedx: [[0. 0. 0. 0.]
 [0. 0. 0. 0.]]
0回目dX: [[125  93 102]
 [102  93  93]]
1回目dX: [[137 102 113]
 [113 102 102]]
dout [[125. 230. 204. 113.]
 [102. 206. 195. 102.]]


# 問題5 パディングの実装

In [14]:
class SimpleConv1d_ver3_1:
    
    def __init__(self,filter_num, filter_length,
                 initializer, optimizer,sigma,
                 opt_params, stride=1, padding=0):
        #self.input_shape= input_shape
        self.filter_num = filter_num
        self.filter_length = filter_length
        print(self.filter_length)
        self.stride = stride
        self.padding = padding
        self.sigma = sigma
        self.initializer = initializer(self.sigma)
        self.optimizer = optimizer(**opt_params)
        
        
        self.params = {}
        self.grads = {}
        self.X = None
        self.col = None
        self.col_W = None
        self.out_size = None
        #self.params['W'] = self.initializer.W(self.filter_num, self.filter_length)
        
        #Wを縦ベクトルで定義する
        self.params['W'] = np.ones((3, 2, 3))#(FN,C,filter_length)
        
        #self.params['B'] = self.initializer.B()
        self.params['B'] = np.array([1, 2, 3]).reshape(1,-1)
        self.dW = np.zeros_like(self.params['W'])
        self.dB = np.zeros_like(self.params['B'])
        self.grads['W'] = self.dW
        self.grads['B'] = self.dB
        
        
    def forward(self,X):
        #batchsize,channel共に１のとき
#         if X.ndim == 1:
#             X = X[np.newaxis,np.newaxis,:]
#         #batchsizeが１のとき
#         if X.ndim == 2:
#             X = X[np.newaxis,:,:]
            
        C,D = X.shape
        self.C = C
        X = np.pad(X,[(0,0),(self.padding,self.padding)],'constant')
        print('pad後x:', X)
        print('Wshape:',self.params['W'].shape)
        self.X = X
        
        
        self.out_size = self.calc_out_size(D, self.filter_length,
                                           self.padding,self.stride)
        print('out_size',self.out_size)
        
        #Wをcolに変換
        col_W = self.params['W'].reshape(-1,self.filter_num)
        self.col_W = col_W
        print('col_Wshape:', col_W.shape)

        self.col = np.zeros((self.out_size, C * self.filter_length))
        print('初期化時colshape', self.col.shape)

        #Xをcolに変換
        for i in range(self.out_size):
            print('x部分shape', X[:, i : i + self.filter_length].shape)
            self.col[i,:] = X[:, i : i + self.filter_length].reshape(1,-1)
            
        print('colshape:', self.col.shape)
        out_col = np.dot(self.col, col_W) + \
        self.params['B']
        out = out_col.T
        print('outshape:', out.shape)
        return out#(FN,out_size)
        
    
    
    def backward(self, dout):
        #shape: dout=(out_size,FN), col=(out_size,filter_length),
        #W=(FN, filter_length)
        dout = dout.T#(out_size, FN)
        col_dX = np.dot(dout, self.col_W.T)
        print('col_dX:', col_dX)
        #(out_size,FN)dot(FN,filter_length*C)
        dX = np.zeros_like(self.X).astype(np.float64)
        print('dxshape',dX.shape)
        #(filter_length * C, out_size)dot(out_size, FN)
        self.grads['W'] = np.dot(self.col.T, dout)
        self.grads['B'] = np.sum(dout,axis=0)
        print('beforedx:', dX)
        
        #col_dXをdXに変換
        col_dX_3d = col_dX.reshape(self.C,self.filter_length, self.out_size)
        for i in range(self.out_size):
            dX[:,i : i + self.filter_length] += col_dX_3d[:,:,i]
            print(str(i) + '回目dX:', dX)
     
        return dX
    
    def padding(self,X,padding):
        return np.pad(X,[0,padding],'constant')
        
        
        
        
    
    def calc_out_size(self, D, filter_length, padding, stride):
        return (D - filter_length + 2 * padding) // stride + 1

In [15]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
b = np.array([1, 2, 3])


filter_num = 3
filter_length = 3
sigma = 0.01
X_sample =np.array([[1, 2, 3, 4], [2, 3, 4, 5]])


conv1d = SimpleConv1d_ver3_1(filter_num=filter_num,
                      filter_length=filter_length, sigma = sigma,
                      initializer=ConvSimpleInitializer, optimizer=SGD,
                 opt_params={'lr' : 0.01}, padding=1)
out = conv1d.forward(X_sample)
print(out)
delta_a = out

#delta_a  = np.array([[16, 22], [17, 23], [18, 24]])
dout = conv1d.backward(delta_a)



print('dout', dout)

3
pad後x: [[0 1 2 3 4 0]
 [0 2 3 4 5 0]]
Wshape: (3, 2, 3)
out_size 4
col_Wshape: (6, 3)
初期化時colshape (4, 6)
x部分shape (2, 3)
x部分shape (2, 3)
x部分shape (2, 3)
x部分shape (2, 3)
colshape: (4, 6)
outshape: (3, 4)
[[ 9. 16. 22. 17.]
 [10. 17. 23. 18.]
 [11. 18. 24. 19.]]
col_dX: [[30. 30. 30. 30. 30. 30.]
 [51. 51. 51. 51. 51. 51.]
 [69. 69. 69. 69. 69. 69.]
 [54. 54. 54. 54. 54. 54.]]
dxshape (2, 6)
beforedx: [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
0回目dX: [[30. 30. 51.  0.  0.  0.]
 [69. 69. 54.  0.  0.  0.]]
1回目dX: [[ 30.  60.  81.  51.   0.   0.]
 [ 69. 138. 123.  54.   0.   0.]]
2回目dX: [[ 30.  60. 111. 102.  51.   0.]
 [ 69. 138. 192. 108.  54.   0.]]
3回目dX: [[ 30.  60. 111. 132. 102.  51.]
 [ 69. 138. 192. 177. 108.  54.]]
dout [[ 30.  60. 111. 132. 102.  51.]
 [ 69. 138. 192. 177. 108.  54.]]


# 問題6 ミニバッチへの対応

In [43]:
class SimpleConv1d_ver3_1:
    
    def __init__(self,filter_num, filter_length,
                 initializer, optimizer,sigma,
                 opt_params, stride=1, padding=0):
        #self.input_shape= input_shape
        self.filter_num = filter_num
        self.filter_length = filter_length
        print(self.filter_length)
        self.stride = stride
        self.padding = padding
        self.sigma = sigma
        self.initializer = initializer(self.sigma)
        self.optimizer = optimizer(**opt_params)
        
        
        self.params = {}
        self.grads = {}
        self.X = None
        self.col = None
        self.col_W = None
        self.out_size = None
        #self.params['W'] = self.initializer.W(self.filter_num, self.filter_length)
        
        #Wを縦ベクトルで定義する
        self.params['W'] = np.ones((3, 2, 3))#(FN,C,filter_length)
        
        #self.params['B'] = self.initializer.B()
        self.params['B'] = np.array([1, 2, 3]).reshape(1,-1)
        self.dW = np.zeros_like(self.params['W'])
        self.dB = np.zeros_like(self.params['B'])
        self.grads['W'] = self.dW
        self.grads['B'] = self.dB
        
        
    def forward(self,X):
        #batchsize,channel共に１のとき
#         if X.ndim == 1:
#             X = X[np.newaxis,np.newaxis,:]
#         #batchsizeが１のとき
#         if X.ndim == 2:
#             X = X[np.newaxis,:,:]
            
        N,C,D = X.shape
        self.C = C
        self.D = D
        self.N = N
        
        X = np.pad(X,[(0,0),(0,0),(self.padding,self.padding)],'constant')
        print('pad後x:', X)
        print('Wshape:',self.params['W'].shape)
        self.X = X
        
        
        self.out_size = self.calc_out_size(D, self.filter_length,
                                           self.padding,self.stride)
        print('out_size',self.out_size)
        
        #Wをcolに変換
        col_W = self.params['W'].reshape(-1,self.filter_num)
        self.col_W = col_W
        print('col_Wshape:', col_W.shape)

        self.col = np.zeros((self.N * self.out_size,
                             self.C * self.filter_length))
        
        print('初期化時colshape', self.col.shape)

        #Xをcolに変換
        for i in range(self.out_size):
            idxes =np.arange(i,self.N * self.out_size,self.out_size)
            print('idxes',idxes)
            print('x部分shape', X[:, :, i : i + self.filter_length].shape)
            self.col[idxes,:] = X[:, :, i : i + self.filter_length].reshape(self.N,
                                                                            -1)
            
        print('self.col',self.col)    
        print('colshape:', self.col.shape)
        out_col = np.dot(self.col, col_W) + \
        self.params['B']
        print('out_col:', out_col)
        out_col = out_col.reshape(self.N, self.out_size, self.filter_num)
        out = out_col.transpose(0,2,1)
        print('outshape:', out.shape)
        return out#(N,FN,out_size)
        
    
    
    def backward(self, dout):
        #shape: dout=(out_size,FN), col=(out_size,filter_length),
        #W=(FN, filter_length)
        #まず、受け取ったdout→(4,3)(N*out, FN)の形に直してから微分する
        dout = dout.transpose(0,2,1)#(N,out_size,FN)
        dout = dout.reshape(-1,self.filter_num)#(N*out_size, FN)
        col_dX = np.dot(dout, self.col_W.T)
        print('col_dX:', col_dX)
        #(out_size,FN)dot(FN,filter_length*C)
        dX = np.zeros_like(self.X).astype(np.float64)
        print('dxshape',dX.shape)
        #(filter_length * C, out_size)dot(out_size, FN)
        self.grads['W'] = np.dot(self.col.T, dout)
        print('gradW:', self.grads['W'])
        self.grads['B'] = np.sum(dout,axis=0)
        print('gradB:', self.grads['B'])
        print('beforedx:', dX)
        
        #col_dXをdXに変換
        col_dX_4d = \
        col_dX.reshape(self.N, self.out_size, self.C, self.filter_length)
        col_dX_4d = col_dX_4d.transpose(0,2,3,1)
        
        for i in range(self.out_size):
            dX[:,:,i : i + self.filter_length] += col_dX_4d[:,:,:,i]
            print(str(i) + '回目dX:', dX)
     
        return dX
    
    def padding(self,X,padding):
        return np.pad(X,[0,padding],'constant')
        
        
        
        
    
    def calc_out_size(self, D, filter_length, padding, stride):
        return (D - filter_length + 2 * padding) // stride + 1

In [44]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
b = np.array([1, 2, 3])


filter_num = 3
filter_length = 3
sigma = 0.01
X_sample =np.arange(1,17,1).reshape(2,2,4)

conv1d = SimpleConv1d_ver3_1(filter_num=filter_num,
                      filter_length=filter_length, sigma = sigma,
                      initializer=ConvSimpleInitializer, optimizer=SGD,
                 opt_params={'lr' : 0.01}, padding=0)
out = conv1d.forward(X_sample)
print(out)
delta_a = out

#delta_a  = np.array([[16, 22], [17, 23], [18, 24]])
dout = conv1d.backward(delta_a)



print('dout', dout)

3
pad後x: [[[ 1  2  3  4]
  [ 5  6  7  8]]

 [[ 9 10 11 12]
  [13 14 15 16]]]
Wshape: (3, 2, 3)
out_size 2
col_Wshape: (6, 3)
初期化時colshape (4, 6)
idxes [0 2]
x部分shape (2, 2, 3)
idxes [1 3]
x部分shape (2, 2, 3)
self.col [[ 1.  2.  3.  5.  6.  7.]
 [ 2.  3.  4.  6.  7.  8.]
 [ 9. 10. 11. 13. 14. 15.]
 [10. 11. 12. 14. 15. 16.]]
colshape: (4, 6)
out_col: [[25. 26. 27.]
 [31. 32. 33.]
 [73. 74. 75.]
 [79. 80. 81.]]
outshape: (2, 3, 2)
[[[25. 31.]
  [26. 32.]
  [27. 33.]]

 [[73. 79.]
  [74. 80.]
  [75. 81.]]]
col_dX: [[ 78.  78.  78.  78.  78.  78.]
 [ 96.  96.  96.  96.  96.  96.]
 [222. 222. 222. 222. 222. 222.]
 [240. 240. 240. 240. 240. 240.]]
dxshape (2, 2, 4)
gradW: [[1534. 1556. 1578.]
 [1742. 1768. 1794.]
 [1950. 1980. 2010.]
 [2366. 2404. 2442.]
 [2574. 2616. 2658.]
 [2782. 2828. 2874.]]
gradB: [208. 212. 216.]
beforedx: [[[0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]]]
0回目dX: [[[ 78.  78.  78.   0.]
  [ 78.  78.  78.   0.]]

 [[222. 222. 222.   0.]
  [222. 222. 222

# 問題7 任意のストライド数

In [45]:
class SimpleConv1d_ver4_1:
    
    def __init__(self,filter_num, filter_length,
                 initializer, optimizer,sigma,
                 opt_params, stride=1, padding=0):
        #self.input_shape= input_shape
        self.filter_num = filter_num
        self.filter_length = filter_length
        print(self.filter_length)
        self.stride = stride
        self.padding = padding
        self.sigma = sigma
        self.initializer = initializer(self.sigma)
        self.optimizer = optimizer(**opt_params)
        
        
        self.params = {}
        self.grads = {}
        self.X = None
        self.col = None
        self.col_W = None
        self.out_size = None
        #self.params['W'] = self.initializer.W(self.filter_num, self.filter_length)
        
        #Wを縦ベクトルで定義する
        self.params['W'] = np.ones((3, 2, 2))#(FN,C,filter_length)
        
        #self.params['B'] = self.initializer.B()
        self.params['B'] = np.array([1, 2, 3]).reshape(1,-1)
        self.dW = np.zeros_like(self.params['W'])
        self.dB = np.zeros_like(self.params['B'])
        self.grads['W'] = self.dW
        self.grads['B'] = self.dB
        
        
    def forward(self,X):
        #batchsize,channel共に１のとき
#         if X.ndim == 1:
#             X = X[np.newaxis,np.newaxis,:]
#         #batchsizeが１のとき
#         if X.ndim == 2:
#             X = X[np.newaxis,:,:]
            
        N,C,D = X.shape
        self.C = C
        self.D = D
        self.N = N
        
        X = np.pad(X,[(0,0),(0,0),(self.padding,self.padding)],'constant')
        print('pad後x:', X)
        print('Wshape:',self.params['W'].shape)
        self.X = X
        
        
        self.out_size = self.calc_out_size(D, self.filter_length,
                                           self.padding,self.stride)
        print('out_size',self.out_size)
        
        #Wをcolに変換
        col_W = self.params['W'].reshape(-1,self.filter_num)
        self.col_W = col_W
        print('col_Wshape:', col_W.shape)

        self.col = np.zeros((self.N * self.out_size,
                             self.C * self.filter_length))
        
        print('初期化時colshape', self.col.shape)

        #Xをcolに変換
        for i in range(self.out_size):
            print('i',i)
            idxes =np.arange(i,self.N * self.out_size,self.out_size)
            print('idxes',idxes)
            filtered_idx = self.stride * i
            print('x部分shape', X[:, :,filtered_idx : filtered_idx+ \
                                self.filter_length].shape)
            self.col[idxes,:] = X[:, :, i : i + self.filter_length].reshape(self.N,
                                                                            -1)
            
        print('self.col',self.col)    
        print('colshape:', self.col.shape)
        out_col = np.dot(self.col, col_W) + \
        self.params['B']
        print('out_col:', out_col)
        out_col = out_col.reshape(self.N, self.out_size, self.filter_num)
        out = out_col.transpose(0,2,1)
        print('outshape:', out.shape)
        return out#(N,FN,out_size)
        
    
    
    def backward(self, dout):
        #shape: dout=(out_size,FN), col=(out_size,filter_length),
        #W=(FN, filter_length)
        #まず、受け取ったdout→(4,3)(N*out, FN)の形に直してから微分する
        dout = dout.transpose(0,2,1)#(N,out_size,FN)
        dout = dout.reshape(-1,self.filter_num)#(N*out_size, FN)
        col_dX = np.dot(dout, self.col_W.T)
        print('col_dX:', col_dX)
        #(out_size,FN)dot(FN,filter_length*C)
        dX = np.zeros_like(self.X).astype(np.float64)
        print('dxshape',dX.shape)
        #(filter_length * C, out_size)dot(out_size, FN)
        self.grads['W'] = np.dot(self.col.T, dout)
        print('gradW:', self.grads['W'])
        self.grads['B'] = np.sum(dout,axis=0)
        print('gradB:', self.grads['B'])
        print('beforedx:', dX)
        
        #col_dXをdXに変換
        col_dX_4d = \
        col_dX.reshape(self.N, self.out_size, self.C, self.filter_length)
        col_dX_4d = col_dX_4d.transpose(0,2,3,1)
        
        for i in range(self.out_size):
            filtered_idx = self.stride * i
            dX[:,:,filtered_idx : filtered_idx + self.filter_length] \
            += col_dX_4d[:,:,:,i]
            print(str(i) + '回目dX:', dX)
     
        return dX
    
    def padding(self,X,padding):
        return np.pad(X,[0,padding],'constant')
        
        
        
        
    
    def calc_out_size(self, D, filter_length, padding, stride):
        print('fl',filter_length)
        print('D',D)
        return (D - filter_length + 2 * padding) // stride + 1

In [46]:
filter_num = 3
filter_length = 2
sigma = 0.01
X_sample =np.arange(1,17,1).reshape(2,2,4)

conv1d = SimpleConv1d_ver4_1(filter_num=filter_num,
                      filter_length=filter_length, sigma = sigma,
                      initializer=ConvSimpleInitializer, optimizer=SGD,
                 opt_params={'lr' : 0.01}, padding=0,stride=2)
out = conv1d.forward(X_sample)
print(out)
delta_a = out

#delta_a  = np.array([[16, 22], [17, 23], [18, 24]])
dout = conv1d.backward(delta_a)



print('dout', dout)

2
pad後x: [[[ 1  2  3  4]
  [ 5  6  7  8]]

 [[ 9 10 11 12]
  [13 14 15 16]]]
Wshape: (3, 2, 2)
fl 2
D 4
out_size 2
col_Wshape: (4, 3)
初期化時colshape (4, 4)
i 0
idxes [0 2]
x部分shape (2, 2, 2)
i 1
idxes [1 3]
x部分shape (2, 2, 2)
self.col [[ 1.  2.  5.  6.]
 [ 2.  3.  6.  7.]
 [ 9. 10. 13. 14.]
 [10. 11. 14. 15.]]
colshape: (4, 4)
out_col: [[15. 16. 17.]
 [19. 20. 21.]
 [47. 48. 49.]
 [51. 52. 53.]]
outshape: (2, 3, 2)
[[[15. 19.]
  [16. 20.]
  [17. 21.]]

 [[47. 51.]
  [48. 52.]
  [49. 53.]]]
col_dX: [[ 48.  48.  48.  48.]
 [ 60.  60.  60.  60.]
 [144. 144. 144. 144.]
 [156. 156. 156. 156.]]
dxshape (2, 2, 4)
gradW: [[ 986. 1008. 1030.]
 [1118. 1144. 1170.]
 [1514. 1552. 1590.]
 [1646. 1688. 1730.]]
gradB: [132. 136. 140.]
beforedx: [[[0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]]]
0回目dX: [[[ 48.  48.   0.   0.]
  [ 48.  48.   0.   0.]]

 [[144. 144.   0.   0.]
  [144. 144.   0.   0.]]]
1回目dX: [[[ 48.  48.  60.  60.]
  [ 48.  48.  60.  60.]]

 [[144. 144. 156. 156.]
  [144

# 問題8 学習と推定

In [71]:
class SimpleConv1d_completed:
    
    def __init__(self,filter_num, filter_length,
                optimizer,sigma,
                 opt_params, C,stride=1, padding=0):
        #self.input_shape= input_shape
        self.filter_num = filter_num
        self.filter_length = filter_length
        print(self.filter_length)
        self.stride = stride
        self.padding = padding
        self.sigma = sigma
        self.C = C
        #self.initializer = initializer(self.sigma)
        self.optimizer = optimizer
        
        
        self.params = {}
        self.grads = {}
        self.X = None
        self.col = None
        self.col_W = None
        self.out_size = None
        #self.params['W'] = self.initializer.W(self.filter_num, self.filter_length)
        
        #Wを縦ベクトルで定義する
        self.params['W'] = np.random.randn(self.filter_num,self.C, 
                                    self.filter_length) * self.sigma
        #(FN,C,filter_length)
        
        #self.params['B'] = self.initializer.B()
        self.params['B'] = np.random.randn(self.filter_num).reshape(1,-1) *\
        self.sigma
        self.dW = np.zeros_like(self.params['W'])
        self.dB = np.zeros_like(self.params['B'])
        self.grads['W'] = self.dW
        self.grads['B'] = self.dB
        
        
    def forward(self,X):
        #batchsize,channel共に１のとき
#         if X.ndim == 1:
#             X = X[np.newaxis,np.newaxis,:]
#         #batchsizeが１のとき
#         if X.ndim == 2:
#             X = X[np.newaxis,:,:]
            
        N,C,D = X.shape
        self.D = D
        self.N = N
        
        X = np.pad(X,[(0,0),(0,0),(self.padding,self.padding)],'constant')
#         print('pad後x:', X)
#         print('Wshape:',self.params['W'].shape)
        self.X = X
        
        
        self.out_size = self.calc_out_size(D, self.filter_length,
                                           self.padding,self.stride)
#         print('out_size',self.out_size)
        
        #Wをcolに変換
        col_W = self.params['W'].reshape(-1,self.filter_num)
        self.col_W = col_W
#         print('col_Wshape:', col_W.shape)

        self.col = np.zeros((self.N * self.out_size,
                             self.C * self.filter_length))
        
#         print('初期化時colshape', self.col.shape)

        #Xをcolに変換
        for i in range(self.out_size):
#             print('i',i)
            idxes =np.arange(i,self.N * self.out_size,self.out_size)
#             print('idxes',idxes)
            filtered_idx = self.stride * i
#             print('x部分shape', X[:, :,filtered_idx : filtered_idx+ \
#                                 self.filter_length].shape)
            self.col[idxes,:] = X[:, :, i : i + self.filter_length].reshape(self.N,
                                                                            -1)
            
#         print('self.col',self.col)    
#         print('colshape:', self.col.shape)
        out_col = np.dot(self.col, col_W) + \
        self.params['B']
#         print('out_col:', out_col)
        out_col = out_col.reshape(self.N, self.out_size, self.filter_num)
        out = out_col.transpose(0,2,1)
#         print('outshape:', out.shape)
        return out#(N,FN,out_size)
        
    
    
    def backward(self, dout):
        #shape: dout=(out_size,FN), col=(out_size,filter_length),
        #W=(FN, filter_length)
        #まず、受け取ったdout→(4,3)(N*out, FN)の形に直してから微分する
        dout = dout.transpose(0,2,1)#(N,out_size,FN)
        dout = dout.reshape(-1,self.filter_num)#(N*out_size, FN)
        col_dX = np.dot(dout, self.col_W.T)
#         print('col_dX:', col_dX)
        #(out_size,FN)dot(FN,filter_length*C)
        dX = np.zeros_like(self.X).astype(np.float64)
#         print('dxshape',dX.shape)
        #(filter_length * C, out_size)dot(out_size, FN)
        self.grads['W'] = np.dot(self.col.T, dout)
#         print('gradW:', self.grads['W'])
        self.grads['B'] = np.sum(dout,axis=0)
#         print('gradB:', self.grads['B'])
#         print('beforedx:', dX)
        
        #col_dXをdXに変換
        col_dX_4d = \
        col_dX.reshape(self.N, self.out_size, self.C, self.filter_length)
        col_dX_4d = col_dX_4d.transpose(0,2,3,1)
        
        for i in range(self.out_size):
            filtered_idx = self.stride * i
            dX[:,:,filtered_idx : filtered_idx + self.filter_length] \
            += col_dX_4d[:,:,:,i]
#             print(str(i) + '回目dX:', dX)
     
        return dX
    
    def padding(self,X,padding):
        return np.pad(X,[0,padding],'constant')
        
        
        
        
    
    def calc_out_size(self, D, filter_length, padding, stride):
#         print('fl',filter_length)
#         print('D',D)
        return (D - filter_length + 2 * padding) // stride + 1

In [48]:
import numpy as np
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split


(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(-1,784)
X_test = X_test.reshape(-1,784)

X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
print(X_train.max())
print(X_train.min())
print(X_train.shape)
print(y_train.shape)


y_train_one_hot = to_categorical(y_train, num_classes=10)
print(y_train.shape)
print(y_train_one_hot.shape) # (60000, 10)
print(y_train_one_hot.dtype) # float64
print(type(y_train_one_hot))


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot,
                                                 test_size=0.2)

1.0
0.0
(60000, 784)
(60000,)
(60000,)
(60000, 10)
float32
<class 'numpy.ndarray'>


In [72]:
network = ScratchSimpleNeuralNetrowkClassifier(input_shape=784, n_classes=10, n_nodes1=400,
                                              n_nodes2 = 200, initializer=SimpleInitializer, optimizer=SGD,sigma = 0.01, opt_params={'lr' : 0.01})


trainer = Trainer(model=network, n_epochs=5, batch=20)
trainer.fit(X_train,y_train)

3
out_size: 782
affine_input_shape: 2346
*********1エポック*********
loss: 2.308711457179881
*********2エポック*********
loss: 2.307944453203697
*********3エポック*********
loss: 2.3071999207398917
*********4エポック*********
loss: 2.3064799426649785
*********5エポック*********
loss: 2.3057859117647914


In [68]:
from collections import OrderedDict

class ScratchSimpleNeuralNetrowkClassifier:
    def __init__(self, input_shape, n_classes, n_nodes1, n_nodes2,
                 initializer,optimizer, sigma,opt_params):
        self.input_shape = input_shape
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.sigma = sigma 
        self.initializer = initializer(sigma)
        self.optimizer_0 = optimizer(**opt_params)
        self.optimizer_1 = optimizer(**opt_params)
        self.optimizer_2 = optimizer(**opt_params)
        self.optimizer_3 = optimizer(**opt_params)
        #self.optimizer = optimizer(self.alpha, self.batch)
        self.conv_layer = SimpleConv1d_completed(
        filter_num=3, filter_length=3,
                 optimizer=self.optimizer_0,sigma=0.01,
                 opt_params={'lr' : 0.01}, C=1)
        #out=(N,FN,out_size)
        #out_sizeを計算：
        self.out_size = self.conv_layer.calc_out_size(self.input_shape, filter_length=3, padding=0, stride=1)
        print('out_size:', self.out_size)
        affine_input_shape = 3 * self.out_size
        print('affine_input_shape:', affine_input_shape)
        self.layers = OrderedDict()
   
        self.layers['Affine1'] = FC(affine_input_shape, self.n_nodes1,
                                   self.initializer, self.optimizer_1)
        self.layers['Sigmoid1'] = Sigmoid()
        self.layers['Affine2'] = FC(self.n_nodes1, self.n_nodes2,
                                   self.initializer, self.optimizer_2)
        self.layers['Sigmoid2'] = Sigmoid()
        self.layers['Affine3'] = FC(self.n_nodes2, n_classes,self.initializer,
                                   self.optimizer_3)
        self.last_layer = SoftmaxWithLoss()
        
        
    def forward(self,X,y):
        
        self.N,self.D = X.shape
        X = X[:,np.newaxis,:]
        X = self.conv_layer.forward(X)
        N,self.filter_num,self.out_size = X.shape
        X = X.reshape(N,-1)
        
        for layer in self.layers.values():
            X = layer.forward(X)
            
        loss = self.last_layer.forward(X,y)
        return loss
    
    
    def backward(self,dout=1):
        dout = self.last_layer.backward(dout)
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        dout = dout.reshape(self.N,self.filter_num,self.out_size)
        dout = self.conv_layer.backward(dout)
    
    def predict(self,X):
        for layer in self.layers.values():
            X = layer.forward(X)
        return np.argmax(X,axis=1)
        
        
    
    def accuracy(self,X,y):
        y_pred = self.predict(X)
        y = np.argmax(y,axis=1)
        print('y_pred: ', y_pred)
        print('y: ', y)
        return 100 * np.sum(y_pred == y) / len(y)

# ここから前回の読み込み

In [51]:
class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.initializer = initializer
        self.optimizer = optimizer
        
        
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.params = {}
        self.grads = {}
        self.W = self.initializer.W(self.n_nodes1, self.n_nodes2)
        self.B = self.initializer.B(self.n_nodes2)
        self.params['W'] = self.W
        self.params['B'] = self.B
        self.X = None
        self.dW = np.zeros_like(self.W)
        self.dB = np.zeros_like(self.B)
        self.grads['W'] = self.dW
        self.grads['B'] = self.dB
        
        
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """        
        self.X = X
        A = np.dot(X,self.W) + self.B
        return A
    def backward(self, dA):#x=(N,D) w=(D,H),A=(N,H))
#         print('dAの値：', dA)
#         print('Xの値：', self.X)
        dZ = np.dot(dA, self.W.T)
        self.grads['W'][...] = np.dot(self.X.T, dA)
        self.grads['B'][...] = np.sum(dA,axis=0)
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        # 更新
        self = self.optimizer.update(self)
        return dZ

In [52]:
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma):
        self.sigma = sigma
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W :
        """
        W = np.random.randn(n_nodes1, n_nodes2) * self.sigma
        return W
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :
        """
        
        B = np.random.randn(n_nodes2,)
        return B

In [53]:
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
    def update(self, layer):
        grads = layer.grads
        for key in layer.params.keys():
            #print('勾配：', grads[key])
            layer.params[key] -= self.lr * grads[key]
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """
        

In [54]:
class Sigmoid:
    
    def __init__(self):
        self.z = None
        
    def forward(self,x):
        z = 1 / (1 + np.exp(-x))
        self.z = z
        return z
    
    def backward(self,dout):
        return self.z * (1 - self.z) * dout
    

In [55]:
class SoftmaxWithLoss:
    def init(self):
        self.y = None
        self.t = None
    
    def forward(self,a,t):
        self.t = t
        a_max = np.max(a,axis=1).reshape(-1,1)
        a_exp = np.exp(a - a_max)
        a_sum = np.sum(a_exp,axis=1)
        y = a_exp / a_sum.reshape(-1,1)
        self.y = y
        
        loss = self.mean_cross_entropy(y,t)
        return loss
    
    def backward(self,dout=1):
        return (self.y - self.t) / len(self.t)
    
    def mean_cross_entropy(self,y,t,eps=1e-7):
        return - np.sum(t * np.log(y + eps)) / len(t)
        
    

In [56]:
class Trainer:
    
    def __init__(self,model, n_epochs, batch):
        self.model = model
        self.n_epochs = n_epochs
        self.batch = batch
        self.loss_list = []
        
        
        
    def fit(self,X,y):
        
        for i in range(self.n_epochs):
            print('*********' + str(i+1) + 'エポック*********')
            get_mini_batch = GetMiniBatch(X, y, batch_size=20)
            count = 0
            for x_mini, y_mini in get_mini_batch:
                count += 1
                loss = self.model.forward(x_mini,y_mini)
                
                self.model.backward()
                if count == len(get_mini_batch):
                    self.loss_list.append(loss)
                    print('loss:',loss)
        

In [57]:
class GetMiniBatch:
    def __init__(self, X, y, batch_size=20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0] / self.batch_size).astype(np.int)
        #self.stopは作成するバッチサイズ数。１エポック分作成する
        
        
    def __len__(self):
        return self._stop
    
    #指定したバッチ番号を取ってきてくれる
    def __getitem__(self, item):
        p0 = item * self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0 : p1], self._y[p0 : p1]

    
    
    #batchカウンターを初期化する
    def __iter__(self):
        self._counter = 0
        return self
    
    #batchを前から一つずつ取ってくる
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter * self.batch_size
        p1 = self._counter * self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0 : p1], self._y[p0 : p1]